# Predicting Bike Departures and Arrivals for Chicago's Bikesharing Program Divvy

## Problem Overview

Divvy is Chicago's bike sharing system since 2013. Conveniently, Divvy also captures all trip data on its website. https://www.divvybikes.com/system-data. One of the main problems with Divvy is that sometimes on certain hours of the day stations can either get full or empty. This is a pain point for riders who either can't find a bike or can't dock when they have to leading to overtime fees. To overcome this problem, divvy manages a series of rebalancer vans that are moving around the city during the day to pick excess bikes from full stations and then place them at stations where there are few bikes but great demand. This is a manual process. 

The goal of the next few workbooks was to see if Divvy data could be used to predict how many bikes were expected to arrive and depart for each hour of each day and at each of the almost 600 stations scattered across the city. If we could predict beforehand the demand at each station then divvy rebalancer vans could move around the city addressing this demand as this would tell them which stations would get full.

In the next few workbooks, I use divvy data to help build a simple regression model using features engineered from 1) the weather 2) station activity to predict the number of arrivals and departures at each station during each hour of the day.

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
import glob
import itertools

## Data Preprocessing
First I start by reading each of the data files

In [63]:
def read_data(f_names = 'Divvy_data/Divvy_Trips'):
    
    divvy_data = []
    filenames = glob.glob(f_names+'_2016*.csv')
    for file in filenames:
        print('Reading in {0}'.format(file))
        divvy_data.append(pd.read_csv(file,
                                      parse_dates=['starttime', 'stoptime'], 
                                      infer_datetime_format=True))
    
    filenames = glob.glob(f_names+'_2017*.csv')
    for file in filenames:
        print('Reading in {0}'.format(file))
        divvy_data.append(pd.read_csv(file,
                                      parse_dates=['start_time', 'end_time'], 
                                      infer_datetime_format=True).rename(columns={'start_time':'starttime'
                                          ,'end_time':'stoptime'}))

    divvy_data = pd.concat(divvy_data)
    return divvy_data

In [64]:
divvy_data = read_data()

Reading in Divvy_data/Divvy_Trips_2016_Q4.csv
Reading in Divvy_data/Divvy_Trips_2016_Q3.csv
Reading in Divvy_data/Divvy_Trips_2016_Q1.csv
Reading in Divvy_data/Divvy_Trips_2016_05.csv
Reading in Divvy_data/Divvy_Trips_2016_04.csv
Reading in Divvy_data/Divvy_Trips_2016_06.csv
Reading in Divvy_data/Divvy_Trips_2017_Q4.csv
Reading in Divvy_data/Divvy_Trips_2017_Q1.csv
Reading in Divvy_data/Divvy_Trips_2017_Q3.csv
Reading in Divvy_data/Divvy_Trips_2017_Q2.csv


Lets look at the first few rows of the data

In [65]:
divvy_data.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,12979228,2016-12-31 23:57:52,2017-01-01 00:06:44,5076,532,502,California Ave & Altgeld St,258,Logan Blvd & Elston Ave,Customer,NaN,NaN
1,12979227,2016-12-31 23:53:18,2017-01-01 00:08:13,5114,895,195,Columbus Dr & Randolph St,25,Michigan Ave & Pearson St,Customer,NaN,NaN
2,12979226,2016-12-31 23:53:07,2017-01-01 00:08:38,1026,931,195,Columbus Dr & Randolph St,25,Michigan Ave & Pearson St,Customer,NaN,NaN
3,12979225,2016-12-31 23:51:31,2017-01-01 00:07:41,504,970,199,Wabash Ave & Grand Ave,35,Streeter Dr & Grand Ave,Subscriber,Male,1985.0
4,12979224,2016-12-31 23:51:31,2017-01-01 00:07:51,4451,980,199,Wabash Ave & Grand Ave,35,Streeter Dr & Grand Ave,Subscriber,Female,1985.0


Now let's read and view the weather data. This data is gathered at the start of each hour at each of the weather stations in Chicago. For simplicity we will just take the mean of each of the weather station readings

In [66]:
weather_data = pd.read_csv('Beach_Weather_Stations_-_Automated_Sensors.csv', parse_dates=['Measurement Timestamp'],infer_datetime_format=True)
weather_data['hour'] = weather_data['Measurement Timestamp'].apply(lambda x: x.hour)
weather_data['date'] = weather_data['Measurement Timestamp'].dt.date
weather_data['day_of_week'] = weather_data['Measurement Timestamp'].apply(lambda x: x.weekday())
weather_data['month'] = weather_data['Measurement Timestamp'].apply(lambda x: x.month)
weather_data['day_of_month'] = weather_data['Measurement Timestamp'].apply(lambda x: x.day)
weather_data = weather_data.groupby(['date','hour']).mean().reset_index()
weather_data = weather_data.sort_values(by = ['date','hour'], ascending = True)
weather_data.tail()

,date,hour,Air Temperature,Wet Bulb Temperature,Humidity,Rain Intensity,Interval Rain,Total Rain,Precipitation Type,Wind Direction,Wind Speed,Maximum Wind Speed,Barometric Pressure,Solar Radiation,Heading,Battery Life,day_of_week,month,day_of_month
25050,2018-05-17,19.0,8.856667,7.95,82.333333,0.0,0.0,106.35,0.0,127.000000,5.400000,7.300000,996.466667,46.000000,178.5,13.033333,3.0,5.0,17.0
25051,2018-05-17,20.0,9.063333,8.00,81.000000,0.0,0.0,106.35,0.0,121.333333,4.600000,7.166667,996.600000,3.666667,178.5,13.033333,3.0,5.0,17.0
25052,2018-05-17,21.0,9.493333,8.45,81.000000,0.0,0.0,106.35,0.0,123.333333,4.733333,6.433333,996.600000,1.000000,178.5,13.033333,3.0,5.0,17.0
25053,2018-05-17,22.0,10.000000,8.80,86.000000,0.0,0.0,53.40,0.0,16.000000,6.400000,8.600000,996.400000,3.000000,355.0,11.900000,3.0,5.0,17.0
25054,2018-05-17,23.0,10.000000,8.70,84.500000,0.0,0.0,106.35,0.0,16.000000,3.700000,6.250000,996.650000,1.500000,178.5,11.950000,3.0,5.0,17.0


And also at the stations file which tells us the location of each station as well as capacity and date it first started service

In [68]:
stations_data =  pd.read_csv('Divvy_data/Divvy_Stations_2016_Q4.csv', parse_dates=['online_date'],infer_datetime_format=True)
stations_data['online_date'] = pd.to_datetime(stations_data['online_date']).dt.date
stations_data.head()

,id,name,latitude,longitude,dpcapacity,online_date
0,456,2112 W Peterson Ave,41.991178,-87.683593,15,2015-05-12
1,101,63rd St Beach,41.781016,-87.576120,23,2015-04-20
2,109,900 W Harrison St,41.874675,-87.650019,19,2013-08-06
3,21,Aberdeen St & Jackson Blvd,41.877726,-87.654787,15,2013-06-21
4,80,Aberdeen St & Monroe St,41.880420,-87.655599,19,2013-06-26


## Aggregating Data at the Right Level
Since the trips data is at the trip level. We will have to do some aggreagting to get the data at the right level. 
We want to predict for each station and for each day and for each hour the number of departures and arrivals. Because some days, hours and stations will have no arrivals and departures, we will need to first manually create a a base  data table that incorporates every combination of stations, days and hours (stations, days, hour). The following functions achieves this efficiently. (station, days, hour) is one unique prediction in our dataset. 

In [123]:
def create_base(data):
    ##get all unique combinations
    
    divvy_data['departure_hour'] = divvy_data['starttime'].apply(lambda x: x.hour)
    divvy_data['departure_date'] = pd.to_datetime(divvy_data['starttime']).dt.date
    
    stations = data['from_station_id'].unique()
    dates = data['departure_date'].unique()
    hours = data['departure_hour'].unique()

    divvy_agg = np.array(list(itertools.product(stations,dates,hours)))
    divvy_agg = pd.DataFrame(divvy_agg, columns = ['station_id','date','hour'])
    divvy_agg = divvy_agg.sort_values(by = ['station_id','date','hour'])
    divvy_agg.head()
    return divvy_agg

divvy_all_rows = create_base(divvy_data)
divvy_all_rows.head()

,station_id,date,hour
4111895,2,2016-01-01,0
4111894,2,2016-01-01,1
4111893,2,2016-01-01,2
4111892,2,2016-01-01,3
4111891,2,2016-01-01,4


Now let's actually start to aggregate the data by (stations, days, hour) and count the number of arrivals and departures at each station 

In [124]:
def agg_data(divvy_data, weather_data):
    
    divvy_data['departure_hour'] = divvy_data['starttime'].apply(lambda x: x.hour)
    divvy_data['arrival_hour'] = divvy_data['stoptime'].apply(lambda x: x.hour)

    divvy_data['departure_date'] = divvy_data['starttime'].dt.date
    divvy_data['arrival_date'] = divvy_data['stoptime'].dt.date
    
    divvy_depart = divvy_data.groupby(['from_station_name','from_station_id','departure_date','departure_hour'])['trip_id'].size().reset_index(name='trips_departed')
    divvy_arrive = divvy_data.groupby(['to_station_name','to_station_id','arrival_date','arrival_hour'])['trip_id'].size().reset_index(name='trips_arrived')
    
    return divvy_depart, divvy_arrive

divvy_depart, divvy_arrive = agg_data(divvy_data, weather_data)
divvy_depart.head()

,from_station_name,from_station_id,departure_date,departure_hour,trips_departed
0,2112 W Peterson Ave,456,2016-01-05,14,1
1,2112 W Peterson Ave,456,2016-01-06,7,1
2,2112 W Peterson Ave,456,2016-01-07,17,1
3,2112 W Peterson Ave,456,2016-01-14,10,1
4,2112 W Peterson Ave,456,2016-01-15,10,1


In [125]:
divvy_arrive.head()

,to_station_name,to_station_id,arrival_date,arrival_hour,trips_arrived
0,2112 W Peterson Ave,456,2016-01-05,13,1
1,2112 W Peterson Ave,456,2016-01-05,18,1
2,2112 W Peterson Ave,456,2016-01-07,16,1
3,2112 W Peterson Ave,456,2016-01-10,11,1
4,2112 W Peterson Ave,456,2016-01-12,20,1


## Creating a Final Base Table
Now that we have the data aggregated by (stations, days, hour) we can now merge it back to our base table. We can also merge in stations and weather data here so we have all the data in one dataset for feature engineering.

In [126]:
def merge_data(divvy_all_rows, divvy_depart, divvy_arrive, stations_data):
    
    divvy_all_rows = divvy_all_rows.merge(divvy_depart.rename(columns={'from_station_id':'station_id'
                                          ,'departure_date':'date'
                                          ,'departure_hour':'hour'})
                                          ,on =['station_id','date','hour']
                                          ,how='left')
    
    divvy_all_rows = divvy_all_rows.merge(divvy_arrive.rename(columns={'to_station_id':'station_id'
                                      ,'arrival_date':'date'
                                      ,'arrival_hour':'hour'})
                                      ,on =['station_id','date','hour']
                                      ,how='left')
    
    divvy_all_rows = divvy_all_rows.merge(weather_data
                                  ,on =['date','hour']
                                  ,how='left')
    
    stations_data = stations_data.filter(['id', 'latitude', 'longitude', 'dpcapacity','online_date']).rename(columns ={'id':'station_id'})
    
    divvy_all_rows = divvy_all_rows.merge(stations_data
                                  ,on =['station_id']
                                  ,how='inner')
    
    divvy_all_rows = divvy_all_rows.drop(columns =['from_station_name', 'to_station_name'])
        
    return divvy_all_rows

divvy_merged = merge_data(divvy_all_rows, divvy_depart, divvy_arrive, stations_data)

Lets look at a few rows of the final dataset we processed. All the data points we need are now in one dataset.
Just one last thing. Some stations were installed in the middle of 2016 or 2017. These stations therefore do not have the full set of data for the entire time period I am looking at. These days then will have no data for such stations. 
For this reason we want to exclude days for a station if the station weren't live at that date. After doing this I save the data so I can move to feature enginerring.

In [127]:
divvy_merged = divvy_merged[divvy_merged['date'] > divvy_merged['online_date']]
divvy_merged = divvy_merged.fillna(0)

print('The size of the final dataset (rows, columns)')
print(divvy_merged.shape)
print('The number of rows in the final dataset: ' + str(divvy_merged.shape[0]))
print('The number of stations in the final dataset: ' + str(len(divvy_merged['station_id'].unique())))

The size of the final dataset (rows, columns)
(9723576, 26)
The number of rows in the final dataset: 9723576
The number of stations in the final dataset: 581


In [129]:
divvy_merged.iloc[5000:5010,]

,station_id,date,hour,trips_departed,trips_arrived,Air Temperature,Wet Bulb Temperature,Humidity,Rain Intensity,Interval Rain,...,Solar Radiation,Heading,Battery Life,day_of_week,month,day_of_month,latitude,longitude,dpcapacity,online_date
5000,2,2016-07-27,8,4.0,0.0,25.900000,21.65,66.333333,0.0,0.0,...,214.000000,179.0,13.033333,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5001,2,2016-07-27,9,5.0,1.0,26.130000,22.20,66.666667,0.0,0.0,...,322.333333,179.0,13.033333,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5002,2,2016-07-27,10,0.0,3.0,25.650000,22.90,72.000000,0.0,0.0,...,305.000000,356.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5003,2,2016-07-27,11,0.0,2.0,25.880000,23.00,71.500000,0.0,0.0,...,364.000000,355.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5004,2,2016-07-27,12,0.0,0.0,26.140000,23.20,74.000000,0.0,0.0,...,405.500000,355.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5005,2,2016-07-27,13,1.0,5.0,26.835000,23.70,72.000000,0.0,0.0,...,421.500000,354.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5006,2,2016-07-27,14,0.0,4.0,26.380000,23.30,73.500000,0.0,0.0,...,404.500000,354.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5007,2,2016-07-27,15,4.0,3.0,26.090000,22.90,72.500000,0.0,0.0,...,371.000000,354.0,13.500000,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5008,2,2016-07-27,16,4.0,6.0,27.153333,22.75,68.333333,0.0,0.0,...,393.666667,177.0,13.033333,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08
5009,2,2016-07-27,17,9.0,11.0,26.726667,23.00,72.333333,0.0,0.0,...,74.333333,177.5,13.033333,2.0,7.0,27.0,41.872638,-87.623979,35,2015-05-08


In [130]:
divvy_merged.to_csv('divvy_agg_data.csv',index=False)

In [ ]:
'''
##get all 
divvy_transposed = divvy_depart.pivot_table(index=['departure_date', 'departure_hour'], columns='from_station_id', values='trips_departed', fill_value=0)
divvy_transposed = pd.DataFrame(divvy_transposed.to_records())

divvy_transposed_arrive = divvy_arrive.pivot_table(index=['arrival_date', 'arrival_hour'], columns='to_station_id', values='trips_arrived', fill_value=0)
divvy_transposed_arrive = pd.DataFrame(divvy_transposed_arrive.to_records())

divvy_final = numpy.transpose([numpy.tile(x, len(y)), numpy.repeat(y, len(x))])
'''